### Building a socioeconomic score for Montreal census tracts (2016)

This notebook documents the construction of a simple socioeconomic score for census tracts on the Island of Montreal, based on Census 2016 data from Statistics Canada.

The social and demographic variables used include:

 - lone-parent households as a percent of all census families in private households

 - unemployment rate

 - prevalence of low-income based on low-income cut-offs, after-tax

 - educational achievement less than post-secondary for ages 25-64, as a percent of total respondents ages 25-64

 - population density



In [1]:
## install necessary libraries

import pandas as pd
import numpy as np


In [2]:
!pip install jenkspy

     |████████████████████████████████| 51kB 11.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/14/1d/b0/45b9ac586dd190ca029df1965fe9f497327506376d51422475
Successfully built jenkspy


In [3]:
import jenkspy

#### Classifying census tracts by lone-parent household data

Load lone-parent household data, and assign descriptive column headers based on names provided in CHASS_loneparent_data_headers.txt.

In [4]:
lonepar_df = pd.read_csv("CHASS_loneparent_data.csv")

lonepar_df.rename(columns={"COL0": "GEOUID", "COL1":"total_HHs", "COL2":"lonepar_HHs"}, inplace=True)

lonepar_df.head()

,GEOUID,total_HHs,lonepar_HHs
0,4620000.0,1101985.0,203895.0
1,4620001.0,680.0,190.0
2,4620002.0,905.0,265.0
3,4620003.0,1710.0,450.0
4,4620004.0,820.0,210.0


Drop the first row, which is data for all of Montreal. Then calculate lone-parent family households as a percent of all family households.

In [5]:
lonepar_df.drop(0, inplace=True)

lonepar_df["%_lonepar_HHs"] = lonepar_df["lonepar_HHs"] / lonepar_df["total_HHs"]

In [6]:
lonepar_df.head()

,GEOUID,total_HHs,lonepar_HHs,%_lonepar_HHs
1,4620001.0,680.0,190.0,0.279412
2,4620002.0,905.0,265.0,0.292818
3,4620003.0,1710.0,450.0,0.263158
4,4620004.0,820.0,210.0,0.256098
5,4620005.0,795.0,195.0,0.245283


Check for NA values. 

Because it won't make sense to rank census tracts with some NA values, fill the NA values with the mean percent of lone-parent households.

In [7]:
lonepar_df.isna().sum()

GEOUID            0
total_HHs        19
lonepar_HHs      19
%_lonepar_HHs    19
dtype: int64

In [8]:
lonepar_df['%_lonepar_HHs'].fillna(lonepar_df['%_lonepar_HHs'].mean(), inplace=True)
lonepar_df['%_lonepar_HHs'].isnull().sum()

0

Cluster the lone-parent household data based on the Jenks natural breaks classification method. Assign a number (rank) to each cluster and census tract. 

In [9]:
lonepar_values = lonepar_df['%_lonepar_HHs']

lonepar_breaks = jenkspy.jenks_breaks(lonepar_values, nb_class = 5)
print(lonepar_breaks)

[0.03636363636363636, 0.13333333333333333, 0.1791907514450867, 0.22777777777777777, 0.2962962962962963, 0.5169491525423728]


In [10]:
labels = [1, 2, 3, 4, 5]

lonepar_df['lonepar_rank'] = pd.cut(lonepar_df['%_lonepar_HHs'], bins = lonepar_breaks, \
                                   labels = labels, include_lowest=True)

lonepar_df.head()

,GEOUID,total_HHs,lonepar_HHs,%_lonepar_HHs,lonepar_rank
1,4620001.0,680.0,190.0,0.279412,4
2,4620002.0,905.0,265.0,0.292818,4
3,4620003.0,1710.0,450.0,0.263158,4
4,4620004.0,820.0,210.0,0.256098,4
5,4620005.0,795.0,195.0,0.245283,4


Check how many census tracts have been assigned to each rank.

In [11]:
lonepar_df['lonepar_rank'].value_counts()

3    281
2    275
4    188
1    179
5     47
Name: lonepar_rank, dtype: int64

Note that the ranking of % lone-parent households is from low to high.

1 = census tracts with a low percentage of lone-parent households <br>
5 = census tracts with a high percentage of lone-parent households

Save the rankings in their own dataframe, to be merged into a df with ranks from all other variables later on. 

In [12]:
lonepar_classed = lonepar_df[['GEOUID', 'lonepar_rank']].copy()

#### Classifying census tracts by educational achievement data

Load education data, and assign descriptive column headers based on names provided in CHASS_education_data_headers.txt.

In [13]:
edu_df = pd.read_csv("CHASS_education_data.csv")
edu_df.head()

,COL0,COL1,COL2,COL3
0,4620000.0,2224010.0,250525.0,407255.0
1,4620001.0,1590.0,265.0,310.0
2,4620002.0,1955.0,345.0,415.0
3,4620003.0,3555.0,590.0,800.0
4,4620004.0,1920.0,280.0,455.0


Rename columns. Drop the first row. 

In [14]:
edu_df.rename(columns={"COL0": "GEOUID", "COL1": "n_respondents", "COL2":"n_noDiploma", \
                       "COL3":"diploma"}, inplace=True)

In [15]:
edu_df.drop(0, inplace=True)

edu_df.head()

,GEOUID,n_respondents,n_noDiploma,diploma
1,4620001.0,1590.0,265.0,310.0
2,4620002.0,1955.0,345.0,415.0
3,4620003.0,3555.0,590.0,800.0
4,4620004.0,1920.0,280.0,455.0
5,4620005.0,1810.0,305.0,385.0


Find number of respondents who have a high school diploma or less as a percent of total respondents.

In [16]:
edu_df["%_diplomaOrLess"] = (edu_df['n_noDiploma'] + edu_df['diploma']) / edu_df['n_respondents']

In [17]:
edu_df.head()

,GEOUID,n_respondents,n_noDiploma,diploma,%_diplomaOrLess
1,4620001.0,1590.0,265.0,310.0,0.361635
2,4620002.0,1955.0,345.0,415.0,0.388747
3,4620003.0,3555.0,590.0,800.0,0.390999
4,4620004.0,1920.0,280.0,455.0,0.382812
5,4620005.0,1810.0,305.0,385.0,0.381215


Check for NA values and fill NAs with the mean.

In [18]:
edu_df['%_diplomaOrLess'].isna().sum()

19

In [19]:

edu_df['%_diplomaOrLess'].fillna(edu_df['%_diplomaOrLess'].mean(), inplace=True)
edu_df['%_diplomaOrLess'].isna().sum()


0

Classify the data using Jenks natural breaks. 

In [20]:
edu_values = edu_df['%_diplomaOrLess']

edu_breaks = jenkspy.jenks_breaks(edu_values, nb_class=5)

print(edu_breaks)


[0.02586206896551724, 0.17257683215130024, 0.2582236842105263, 0.34275618374558303, 0.43453237410071943, 0.7446808510638298]


In [21]:
labels = [1, 2, 3, 4, 5]

edu_df['edu_rank'] = pd.cut(edu_df['%_diplomaOrLess'], bins = edu_breaks, \
                           labels = labels, include_lowest=True)


Check how many census tracts have been assigned to each rank.

In [22]:
edu_df['edu_rank'].value_counts()

3    309
4    230
2    200
1    158
5     73
Name: edu_rank, dtype: int64

Note that the rank of census tracts by % with low educational achievement is from low to high.

1 = a small percentage of respondents have only a high school diploma or less <br> 
5 = a large percentage of respondents have only a high school diploma or less

Save the rankings in a new df to be used later on.

In [23]:
edu_classed = edu_df[['GEOUID', 'edu_rank']].copy()

#### Classifying census tracts by rates of unemployment and prevalence of low-income households

Load the unemployment and low-income rate data. <br>Assign descriptive column headers based on names provided in CHASS_unemployment_and_lowincome_data_headers.txt. <br>Drop the first row, which is a combined total for all of Montreal. <br> Break into two dataframes to be treated one at a time.

In [24]:
df = pd.read_csv("CHASS_unemploy_and_lowincome_data.csv")


In [25]:
df.rename(columns = {"COL7":"%lowIncome", "COL10":"unemployRate"}, inplace=True)

df.drop(0, inplace=True)
df.head()

,GEOUID,%lowIncome,unemployRate
1,4620001.0,16.8,5.0
2,4620002.0,16.6,6.4
3,4620003.0,15.2,9.3
4,4620004.0,14.7,10.2
5,4620005.0,16.3,9.3


Break into two dataframes.

In [26]:
lowInc_df = df[['GEOUID', '%lowIncome']].copy()

unemp_df = df[['GEOUID', 'unemployRate']].copy()

Note these data are already expressed as percentages. 

For each df, fill NAs with the mean.


In [27]:
lowInc_df['%lowIncome'].fillna(lowInc_df['%lowIncome'].mean(), inplace=True)

lowInc_df['%lowIncome'].isna().sum()

0

In [28]:
unemp_df['unemployRate'].fillna(unemp_df['unemployRate'].mean(), inplace=True)

unemp_df['unemployRate'].isna().sum()

0

Working on the unemployment data first, find the natural breaks and classify.

In [29]:
unemp_values = unemp_df['unemployRate']
unemp_breaks = jenkspy.jenks_breaks(unemp_values, nb_class=4)

print(unemp_breaks)

[1.9, 6.2, 9.1, 13.3, 26.9]


In [30]:
labels = [1, 2, 3, 4]

unemp_df['unemp_rank'] = pd.cut(unemp_df['unemployRate'], bins = unemp_breaks, \
                               labels=labels, include_lowest=True)

unemp_df['unemp_rank'].value_counts()

2    359
1    351
3    206
4     54
Name: unemp_rank, dtype: int64

Note that the ranking of unemployment is from low to high.

1 = census tracts with a low unemployment rate <br>
5 = census tracts with a high unemployment rate

Save the rankings in their own df to be used later.

In [31]:
unemp_classed = unemp_df[["GEOUID", "unemp_rank"]].copy()

Repeat the steps for the low-income data.

In [32]:
lowInc_values = lowInc_df['%lowIncome']

lowInc_breaks = jenkspy.jenks_breaks(lowInc_values, nb_class = 4)
print(lowInc_breaks)

[0.6, 9.8, 19.7, 33.7, 58.3]


In [33]:
labels = [1, 2, 3, 4]

lowInc_df['lowInc_rank'] = pd.cut(lowInc_df['%lowIncome'], bins=lowInc_breaks, \
                                 labels=labels, include_lowest=True)

lowInc_df['lowInc_rank'].value_counts()

1    425
2    303
3    214
4     28
Name: lowInc_rank, dtype: int64

Note that the ranking of census tracts by low-income rate is from low to high.

1 = census tracts with a low percentage of low-income households <br>
4 = census tracts with a high percentage of low-income households

Save the ranking in a new df to be used later.

In [34]:
lowInc_classed = lowInc_df[['GEOUID','lowInc_rank']].copy()

#### Classifying census tracts by population density

Load the population density data. <br>Drop the first row, which is a combined total for all of Montreal. <br> Rename columns.

In [35]:
pop_df = pd.read_csv("CHASS_pop_density_data.csv")
pop_df.head()

,geo_uidSTR,pop_per_sqkm,pop_total_2016
0,4620000.0,890.2,4098927.0
1,4620001.0,5710.0,2638.0
2,4620002.0,9029.3,3516.0
3,4620003.0,8612.2,6373.0
4,4620004.0,7087.7,3176.0


In [36]:
pop_df.rename(columns={"geo_uidSTR": "GEOUID"}, inplace=True)

In [37]:
pop_df.drop(0, inplace=True)

Fill NAs in pop_per_sq_km with the mean population density.

In [38]:
pop_df['pop_per_sqkm'].fillna(pop_df['pop_per_sqkm'].mean(), inplace=True)
pop_df['pop_per_sqkm'].isna().sum()

0

Find the natural breaks in pop_per_sqkm and classify.

In [39]:
pop_values = pop_df['pop_per_sqkm']

pop_breaks = jenkspy.jenks_breaks(pop_values, nb_class=4)
print(pop_breaks)



[0.0, 4618.4, 10616.1, 22856.1, 50277.6]


In [40]:
labels = [1,2,3,4]

pop_df['pop_rank'] = pd.cut(pop_df['pop_per_sqkm'], bins=pop_breaks, \
                          labels=labels, include_lowest=True)

pop_df['pop_rank'].value_counts()

1    556
2    252
3    157
4      5
Name: pop_rank, dtype: int64

Save the ranking in a new df for use later.
Note that population density is ranked from low to high, and higher density is considered here as less desirable.

1 = low population density  
4 = high population density  

In [41]:
pop_classed = pop_df[['GEOUID', 'pop_rank']].copy()

#### Collating the rankings from each variable into one dataframe

Set the index for all dfs on the column 'GEOUID' so that they can be joined.

In [42]:
pop_classed = pop_classed.set_index('GEOUID')
lowInc_classed = lowInc_classed.set_index('GEOUID')
unemp_classed = unemp_classed.set_index('GEOUID')
edu_classed = edu_classed.set_index('GEOUID')
lonepar_classed = lonepar_classed.set_index('GEOUID')

Join all the dfs.

In [43]:
from functools import reduce

In [44]:
dfs = [pop_classed, lowInc_classed, unemp_classed, edu_classed, lonepar_classed]

rank_merged = reduce(lambda left,right: pd.merge(left,right,on=['GEOUID'],
                                                how='outer'), dfs)



In [45]:
rank_merged.head()

,pop_rank,lowInc_rank,unemp_rank,edu_rank,lonepar_rank
GEOUID,,,,,
4620001.0,2,2,1,4,4
4620002.0,2,2,2,4,4
4620003.0,2,2,3,4,4
4620004.0,2,2,3,4,4
4620005.0,2,2,3,4,4


Create a new column for the sum of the rankings, and one for the sum divided by the highest possible score (5 + 5 + 4 + 4 + 4 = 22).  
Invert the score so that low scores mean low socioeconomic status and higher scores mean higher status.

In [46]:
rank_merged['sum'] = rank_merged.sum(axis=1)

In [47]:
rank_merged['score'] = 1 - (rank_merged['sum']/22) 

In [49]:
rank_merged['score'] = rank_merged['score'].round(decimals=2)

In [50]:
rank_merged.head()

,pop_rank,lowInc_rank,unemp_rank,edu_rank,lonepar_rank,sum,score
GEOUID,,,,,,,
4620001.0,2,2,1,4,4,13.0,0.41
4620002.0,2,2,2,4,4,14.0,0.36
4620003.0,2,2,3,4,4,15.0,0.32
4620004.0,2,2,3,4,4,15.0,0.32
4620005.0,2,2,3,4,4,15.0,0.32


In [52]:
rank_merged['score'].describe()

count    970.000000
mean       0.504907
std        0.153200
min        0.050000
25%        0.410000
50%        0.500000
75%        0.640000
max        0.770000
Name: score, dtype: float64

Bring back the total population from pop_df to check that scores have been derived from a population of reasonable size.

In [56]:
dfs2 = [rank_merged, pop_df]

scoreAndPop = reduce(lambda left,right: pd.merge(left,right,on=['GEOUID'],
                                                how='outer'), dfs2)

In [57]:
scoreAndPop.head()

,GEOUID,pop_rank_x,lowInc_rank,unemp_rank,edu_rank,lonepar_rank,sum,score,pop_per_sqkm,pop_total_2016,pop_rank_y
0,4620001.0,2,2,1,4,4,13.0,0.41,5710.0,2638.0,2
1,4620002.0,2,2,2,4,4,14.0,0.36,9029.3,3516.0,2
2,4620003.0,2,2,3,4,4,15.0,0.32,8612.2,6373.0,2
3,4620004.0,2,2,3,4,4,15.0,0.32,7087.7,3176.0,2
4,4620005.0,2,2,3,4,4,15.0,0.32,5418.8,3060.0,2


In [58]:
scoreAndPop.drop(['pop_rank_y'], axis=1, inplace=True)

Look at the census tracts with the smallest populations. (I initially checked the 20 smallest and decided that the smallest 17 were too small to use). 

In [65]:
too_small = scoreAndPop.nsmallest(17, 'pop_total_2016')
too_small

,GEOUID,pop_rank_x,lowInc_rank,unemp_rank,edu_rank,lonepar_rank,sum,score,pop_per_sqkm,pop_total_2016
15,4620014.02,1,2,2,3,3,11.0,0.5,0.0,0.0
40,4620040.00,1,2,2,3,3,11.0,0.5,0.0,0.0
76,4620071.00,1,2,2,3,3,11.0,0.5,0.0,0.0
159,4620145.00,1,2,2,3,3,11.0,0.5,0.0,0.0
206,4620189.00,1,2,2,3,3,11.0,0.5,0.0,0.0
250,4620229.00,1,2,2,3,3,11.0,0.5,0.0,0.0
290,4620268.03,1,2,2,3,3,11.0,0.5,0.0,0.0
940,4622006.00,1,2,2,3,3,11.0,0.5,0.0,0.0
965,4622303.00,1,2,2,3,3,11.0,0.5,0.0,0.0
96,4620091.00,1,2,2,3,3,11.0,0.5,5.0,5.0


In [66]:
final_df = scoreAndPop[~scoreAndPop.isin(too_small)].dropna()

The final dataframe includes only census tracts that have a population of 417 or more. 

In [67]:
final_df.nsmallest(17, 'pop_total_2016')

,GEOUID,pop_rank_x,lowInc_rank,unemp_rank,edu_rank,lonepar_rank,sum,score,pop_per_sqkm,pop_total_2016
936,4622002.00,1,1,1,2,1,6.0,0.73,18.8,417.0
180,4620166.00,1,2,1,2,2,8.0,0.64,2091.1,491.0
62,4620061.00,1,3,2,1,2,9.0,0.59,3221.2,699.0
47,4620047.00,2,3,2,2,4,13.0,0.41,6272.2,742.0
588,4620654.00,1,1,1,1,1,5.0,0.77,438.2,744.0
90,4620085.00,1,3,4,5,5,18.0,0.18,662.9,751.0
85,4620080.00,2,3,3,2,2,12.0,0.45,8657.0,780.0
167,4620153.00,1,3,2,3,2,11.0,0.50,2804.6,785.0
56,4620055.02,1,1,1,1,1,5.0,0.77,625.5,822.0
958,4622204.00,1,1,1,4,1,8.0,0.64,182.0,822.0
